In [1]:
import pandas as pd

from collections import Counter
import json

from tfob import TFOb,  get_dss, get_bhsa

In [2]:
BHSA = get_bhsa()

**Locating corpus resources ...**

In [53]:
# Get all the verses in the BHSA database
verses_bhsa = TFOb.all("verse", BHSA).filter(book="Isaiah")

In [54]:
# Initialise a list with the Hebrew Alphabet (including final letters and space)

hebrew_alphabet = [
    'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 
    'י', 'כ', 'ך', 'ל', 'מ', 'ם', 'נ', 'ן', 'ס', 
    'ע', 'פ', 'ף', 'צ', 'ץ', 'ק', 'ר', 'ש', 'ת',
    ' ',
]

# Create a set for faster results

heb_alph_set = set(hebrew_alphabet)

In [55]:
# Test for a function "cleaning" a Hebrew script string from diacritics

clause = "וַיַּ֥רְא אֱלֹהִ֖ים"

clause_cons = [sign for sign in clause if sign in heb_alph_set]

"".join(clause_cons)

'וירא אלהים'

# Create the dataset

In [85]:
# Define functions to initialize the dictionary
verses_dict = {}

# Obtain Hebrew Script without diacritics

def heb_without_diac(clause):
    clause = str(clause.text)
    filtered_signs = [sign for sign in clause if sign in heb_alph_set]
    cleaned_clause = "".join(filtered_signs)
    return cleaned_clause
    

# Function to classify complexity based on the number of clauses
def classify_complexity(verse):
    verse_str = str(verse)
    
    #Check for embedded clauses (if embedded, verse has "complex syntax")
    for clause in verse.to_clauses:
        if str(clause) not in verse_str:
            return "complex"
        
    # If no embedded clause, verse has "simple syntax"
    return "simple"
            

# Function to add or update a sentence in the dictionary
def add_verse_to_dict(verse):
    verse_str = heb_without_diac(verse).strip()
    
    # Assess the verse complexity
    complexity = classify_complexity(verse)
    
    # Create a list with the clauses (as strings)
    clauses = []
    for clause in verse.to_clauses:
        clause = heb_without_diac(clause).strip()
        clauses.append(clause)
    
    # Add the data to the dictionary
    verses_dict[verse_str] = {
        "clauses": clauses,
        "complexity": complexity
    }

In [86]:
# Initialize the dictionary

for verse in verses_bhsa:
    add_verse_to_dict(verse)

In [98]:
# Process the dictionary to remove complexity before json conversion

# Create a new dictionary excluding the "complexity" field
filtered_dict = {verse: details["clauses"] for verse, details in verses_dict.items()}

# Convert to JSON
json_output = json.dumps(filtered_dict, ensure_ascii=False)

json_output

'{"חזון ישעיהו בןאמוץ אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה": ["חזון ישעיהו בןאמוץ", "אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"], "שמעו שמים והאזיני ארץ כי יהוה דבר בנים גדלתי ורוממתי והם פשעו בי": ["שמעו", "שמים", "והאזיני", "ארץ", "כי יהוה דבר", "בנים גדלתי", "ורוממתי", "והם פשעו בי"], "ידע שור קנהו וחמור אבוס בעליו ישראל לא ידע עמי לא התבונן": ["ידע שור קנהו", "וחמור אבוס בעליו", "ישראל לא ידע", "עמי לא התבונן"], "הוי גוי חטא עם כבד עון זרע מרעים בנים משחיתים עזבו אתיהוה נאצו אתקדוש ישראל נזרו אחור": ["הוי גוי חטא עם כבד עון זרע מרעים בנים משחיתים", "עזבו אתיהוה", "נאצו אתקדוש ישראל", "נזרו אחור"], "על מה תכו עוד תוסיפו סרה כלראש לחלי וכללבב דוי": ["על מה תכו עוד", "תוסיפו סרה", "כלראש לחלי", "וכללבב דוי"], "מכףרגל ועדראש איןבו מתם פצע וחבורה ומכה טריה לאזרו ולא חבשו ולא רככה בשמן": ["מכףרגל ועדראש איןבו מתם", "פצע וחבורה ומכה טריה", "לאזרו", "ולא חבשו", "ולא רככה בשמן"], "ארצכם שממה עריכם שרפות אש אדמתכם לנגדכם זרים אכלים אתה ושממה כמה

In [99]:
# Format needed for fine_tuning data

{"messages": 
 [
     {"role": "system", "content": "You are an assistant which identifies clause boundaries in Biblical Hebrew verses."}, 
     {"role": "user", "content": "Biblical Hebrew verse"}, 
     {"role": "assistant", "content": "Structured response."}
]
}

{'messages': [{'role': 'system',
   'content': 'You are an assistant which identifies clause boundaries in Biblical Hebrew verses.'},
  {'role': 'user', 'content': 'Biblical Hebrew verse'},
  {'role': 'assistant', 'content': 'Structured response.'}]}

In [100]:
assistant_role = "You are an assistant which identifies clause boundaries in Biblical Hebrew verses."

#verses_json = {"חזון ישעיהו בןאמוץ אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה": ["חזון ישעיהו בןאמוץ", "אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"],}

jsonl_output = []

for verse, clauses in json_output.items():
    # Create the message structure for each verse
    messages = {
        'messages': [
            {'role': 'system', 'content': assistant_role},
            {'role': 'user', 'content': verse},
            {'role': 'assistant', 'content': json.dumps({verse: clauses}, ensure_ascii=False)}
        ]
    }
    # Append the structure as a JSONL line
    jsonl_output.append(json.dumps(messages, ensure_ascii=False))

# Combine lines into a JSONL string
jsonl_data = "".join(jsonl_output)

# Print or save to file
print(jsonl_data)

AttributeError: 'str' object has no attribute 'items'

---------------------------------

In [14]:
# Visualise the clause of a specific verse

book = "Genesis"
chapter = "1"
verse_num = "12"
verse_bhsa = TFOb.section([book, chapter, verse_num], BHSA)

for clause in verse_bhsa.to_clauses:
    print(clause.to_words.text)

וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ 
מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ 
עֹ֥שֶׂה פְּרִ֛י לְמִינֵ֑הוּ 
אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
וַיַּ֥רְא אֱלֹהִ֖ים 
כִּי־טֹֽוב׃ 


### TODO
- split datasets: training, validation, test
- create one dataset per type (+ training, validation, test), example: transliterated, full_syntax, size 1
- function for formatting the datasets into the proper OpenAI format for fine-tuning (message, role, content, ect)

#### Vocabulary
- fs: full syntax
- ss: simple syntax
- s1, s2, s3: sizes 1, 2, 3 (TBD)
- gcons / heb : transliterated (from gcons ETCBC) and Hebrew script

#### Examples
training_fs_gcons_s1, valid_fs_gcons_s1, test_fs_gcons_s1: training/validation/test full syntax gcons (transliterated) size 1

training_fs_heb_s1, valid_fs_heb_s1, test_fs_heb_s1